In [48]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json

# Set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Set the SPARQL query
query = """
select distinct ?event ?description where {
?event dbp:wikiPageUsesTemplate dbt:France_topics .
?event dbo:abstract ?description .
FILTER(LANG(?description) = "fr").
}
"""

# Set the output format to JSON
sparql.setReturnFormat(JSON)

# Send the SPARQL query to the endpoint
sparql.setQuery(query)
results = sparql.query().convert()

# Save the results as a JSON file
with open("test.json", "w") as f:
    json.dump(results, f, indent=4)


In [8]:
f = open('event.json')
data = json.load(f)

for d in data['results']['bindings'] :
    print(d['event']['value'])
    print(d['description']['value'])
    print('')
  

In [9]:
events = []
# affichage des noms et des identifiants des villes
for result in data["results"]["bindings"]:
    name_event = result['event']['value'].split('/')[-1]
    print(name_event)
    events.append({
        "name": name_event,
        "dbpediaUrl": result['event']['value'],
        "description" : result['description']['value']
    })

In [10]:
with open('event_postTreatment.json', 'w') as outfile:
    json.dump(events, outfile)

In [11]:
for e in events :
    print(e)

# Essai de retrouver les villes dans les descriptions avec du NLP : ne marche pas dès que c'est pas une ville hyper connue ou Américaine

In [12]:
import spacy

nlp = spacy.load('en_core_web_sm')

def extract_cities(text):
    doc = nlp(text)
    cities = []
    for ent in doc.ents:
        if ent.label_ == 'GPE': # GPE: geopolitical entity
            cities.append(ent.text)
    return cities


In [13]:
# Example usage
text = "I love visiting Paris, Berlin and Tokyo."
text2 = "Carloman Ier, né en 751 à Soissons, mort le 4 décembre 771 à Samoussy dans l'Aisne, fut roi des Francs de 768 à 771. Il est le fils et successeur de Pépin le Bref."
cities = extract_cities(text2)
print(cities)  # Output: ['Paris', 'Berlin', 'Tokyo']


# Essai de récupérer les villes à partir du fichier de Yoann

In [14]:
import json

fcities = open('cities.json')
datac = json.load(fcities)

for d in datac :
    print(d['name'])
    print('')

In [15]:
villes = []
# affichage des noms et des identifiants des villes
for d in datac:
    villes.append({
        "name": d['name']
    })

for c in villes:
    #print(c['name'])
    #print('')
    if (c["name"] == "Soissons"):
        print(c["name"])

In [16]:
import re

def find_capital_words(text):
    pattern = r'\b[A-Z][a-z]*\b'  # Matches words beginning with a capital letter
    capital_words = re.findall(pattern, text)
    return capital_words

# Example usage
text = "John Smith is from New York and works at IBM."
capital_words = find_capital_words(text2)
print(capital_words)  # Output: ['John', 'Smith', 'New', 'York', 'IBM']



In [17]:
capital_words = []
for e in events :
    capital_words.append(find_capital_words(e['description']))
for c in capital_words :
    print(c)


In [18]:
capital_words = []

for e in events :
    capital_words.append({
        "name": e['name'],
        "capital_words" : (find_capital_words(e['description'])),
        "description" : e['description']
    })
    
for c in capital_words : 
    print(c['name'])
    print(c['capital_words'])
    print(c['description'])
    print('')

In [19]:
event_to_keep = []

for c in capital_words :
    for i in c['capital_words']:
        for v in villes :   
            if (i== v['name']) and (i!='Francs') :
                event_to_keep.append({
                    "name": c['name'],
                    "description": c['description'],
                    "citie": i
                })



for e in event_to_keep:
    print(e['name'])
    print(e['description'])
    print(e['citie'])
    print('') 

Enlève les doublons

In [20]:
event_to_keep = [dict(t) for t in {tuple(d.items()) for d in event_to_keep}]
print(event_to_keep)

[{'name': 'Cinema_of_France', 'description': "Le cinéma français occupe une place privilégiée dans l'histoire du cinéma, avec des scientifiques comme Joseph Plateau et Étienne-Jules Marey, des inventeurs de premier rang dont les découvertes ont été fondamentales, et avec des pionniers aussi imaginatifs que Alice Guy-Blaché, Émile Reynaud, Louis Lumière, Georges Méliès et Léon Gaumont. Grâce à des financiers autodidactes de la trempe de Charles Pathé, il a contribué au développement de l'industrie mondiale du cinéma. En plus de son propre développement, le cinéma français a permis à de nombreux artistes de l'Europe et du monde entier de s'exprimer. Des réalisateurs célèbres, tels que Krzysztof Kieślowski, Walerian Borowczyk, Andrzej Żuławski, Gaspar Noé, Edgardo Cozarinsky, Alexandre Alexeieff, Anatole Litvak, Michael Haneke, Paul Verhoeven, Otar Iosseliani, John Berry, Roman Polanski, et Maria Koleva, sont comptés parmi les réalisateurs importants du cinéma français. Inversement, des r

In [21]:
with open('event_to_keep.json', 'w') as outfile:
    json.dump(event_to_keep, outfile, indent=4)

# Ajout des dates

In [38]:
# Regular expression to match dates of the form '19XX' or '20XX'
#pattern = r'\b((10|11|12|13|14|15|16|17|18|19|20)\d{2}|(1|2|3|4|5|6|7|8)\d{2}|(X{0,3})(IX|IV|V?I{0,3})\b)'
#pattern = r'\b((20[0-1][0-9]|2[0-9]{2}|1[0-9]{3}|[1-9][0-9]{0,2})|((X{0,3})(IX|IV|V?I{0,3}))|[IVX]{1,4}|([1-9][0-9]{0,2})0|[1-9][0-9]{0,2}0)\b'
#pattern = r'\b((20[0-1][0-9]|2[0-9]{2}|1[0-9]{3}|[1-9][0-9]{0,2})(st|nd|rd|th)?\s+century|((X{0,3})(IX|IV|V?I{0,3}))(st|nd|rd|th)?\s+century|[IVX]{1,4}(st|nd|rd|th)?\s+century|([1-9][0-9]{0,2})0s|[1-9][0-9]{0,2}0)\b'
pattern_dates = r"\b(0|[1-9]\d{0,3}|1\d{3}|20[0-2]\d|2023)\b"
pattern_centuries = r"\b((?:I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX|XXI))\b"

dated_events = []
# Find all matches of the regular expression in the sentence
for e in event_to_keep:
    
    matches_dates = re.findall(pattern_dates, e['description'])
    matches_centuries = re.findall(pattern_centuries, e['description'])
    if (len(matches_centuries)!=0) and (len(matches_dates)!=0):
        print(e['name'])
        print(e['description'])
        print(e['citie'])
        print(matches_dates)
        print(matches_centuries)
        list(set(matches_dates))
        list(set(matches_centuries))
        print(matches_dates)
        print(matches_centuries)
        
        dates = matches_dates+matches_centuries
        dates = ' '.join(dates)
        print(dates)
        dated_events.append({
                    "name": e['name'],
                    "description": e['description'],
                    "citie": e['citie'],
                    "date": dates
                })
    
    print('') 

print(len(dated_events))






Louis_IV_of_France
Louis IV, dit « d'Outremer » (né entre septembre 920 et septembre 921 – 10 septembre 954, Reims), fils de Charles III le Simple et d'Edwige de Wessex, est un roi des Francs (936-954) de la dynastie carolingienne. Après la déchéance en 922 de son père le roi Charles III le Simple, sa mère et le prince Louis, âgé de deux ans, se réfugient en Angleterre (d’où son surnom d’Outremer), à la cour de son grand-père maternel Édouard l'Ancien, puis à celle de son oncle Æthelstan, roi de Wessex. Devenu l'héritier carolingien par la mort en captivité de Charles III (929), il est rappelé d'Angleterre par le puissant marquis de Neustrie Hugues le Grand afin de succéder au roi Raoul mort au début de l'année 936, laquelle marque alors le retour de la dynastie carolingienne. Son règne riche en rebondissements est avant tout connu par les Annales de Flodoard puis plus tardivement par les Histoires de Richer de Reims. Une fois au pouvoir, Louis d'Outremer souhaite s'éloigner d'Hugu

In [39]:
for e in dated_events:
    print(e['name'])
    print(e['description'])
    print(e['citie'])
    print(e['date'])
    print('') 

Louis_IV_of_France
Louis IV, dit « d'Outremer » (né entre septembre 920 et septembre 921 – 10 septembre 954, Reims), fils de Charles III le Simple et d'Edwige de Wessex, est un roi des Francs (936-954) de la dynastie carolingienne. Après la déchéance en 922 de son père le roi Charles III le Simple, sa mère et le prince Louis, âgé de deux ans, se réfugient en Angleterre (d’où son surnom d’Outremer), à la cour de son grand-père maternel Édouard l'Ancien, puis à celle de son oncle Æthelstan, roi de Wessex. Devenu l'héritier carolingien par la mort en captivité de Charles III (929), il est rappelé d'Angleterre par le puissant marquis de Neustrie Hugues le Grand afin de succéder au roi Raoul mort au début de l'année 936, laquelle marque alors le retour de la dynastie carolingienne. Son règne riche en rebondissements est avant tout connu par les Annales de Flodoard puis plus tardivement par les Histoires de Richer de Reims. Une fois au pouvoir, Louis d'Outremer souhaite s'éloigner d'Hugues l

In [40]:
with open('dated_events.json', 'w') as outfile:
    json.dump(dated_events, outfile, indent=4)

In [41]:
print(len(event_to_keep))
print(len(dated_events))

191
55


In [61]:
import json

datas = open('dated_events_v2.json')
datac = json.load(datas)
print(len(datac))
for d in datac :
    print(d['name'])
    print(d['description'])
    print(d['city'])
    print(d['date'])
    print('')

38
Art_francais
La peinture française est considérée comme une des grandes écoles de peinture par son influence, son histoire et ses productions, aux côtés de la peinture italienne, de la peinture flamande et de la . Les premières manifestations de la peinture sur le territoire français s'observent dans l'art préhistorique des grottes ornées, comme celles de Chauvet vers 35 000 ans AP, Cosquer, Lascaux, Pech Merle, Niaux ou Font-de-Gaume, où sont déjà employé la polychromie, l'estompe, la réserve et même les prémices de la perspective, puis à l'époque romaine avec quelques peintures murales. Au Moyen Âge, on trouve des peintures murales dans les édifices religieux comme Saint-Savin-sur-Gartempe, Nohant-Vic, Tavant, Brinay, Berzé-la-Ville, Montoire-sur-le-Loir ou les cathédrales de Cahors, d'Auxerre ou d'Albi, mais surtout dans l'art du vitrail comme à la cathédrale de Chartres, de Paris ou de Bourges ou la Sainte-Chapelle à Paris, etc., des enluminures dans les manuscrits depuis l'époq

In [57]:
import re

# Example sentence
sentence = "Le français est parlé, en 2022, sur tous les continents par environ 321 millions de personnes : 235 millions l'emploient quotidiennement, et 90 millions en sont des locuteurs natifs."

# Regular expression to match numbers followed by "millions"
pattern = r"\b\d+\s+millions\b"

# Find all matches of the pattern in the sentence
matches = re.findall(pattern, sentence)

# Print the matches
print(matches)

['321 millions', '235 millions', '90 millions']


In [60]:
# Regular expression to match numbers followed by "millions"
pattern = r"\b\d+\s+millions\b"
pattern_dates = r"\b(0|[1-9]\d{0,3}|1\d{3}|20[0-2]\d|2023)\b"

for d in datac :
    matches = re.findall(pattern, d['description'])
    for m in matches :        
        match = re.findall(pattern_dates, m)
    print(match)
    
    print(d['name'])
    print(d['description'])
    print(d['city'])
    print(d['date'])
    print('')
    

1
Art_francais
La peinture française est considérée comme une des grandes écoles de peinture par son influence, son histoire et ses productions, aux côtés de la peinture italienne, de la peinture flamande et de la . Les premières manifestations de la peinture sur le territoire français s'observent dans l'art préhistorique des grottes ornées, comme celles de Chauvet vers 35 000 ans AP, Cosquer, Lascaux, Pech Merle, Niaux ou Font-de-Gaume, où sont déjà employé la polychromie, l'estompe, la réserve et même les prémices de la perspective, puis à l'époque romaine avec quelques peintures murales. Au Moyen Âge, on trouve des peintures murales dans les édifices religieux comme Saint-Savin-sur-Gartempe, Nohant-Vic, Tavant, Brinay, Berzé-la-Ville, Montoire-sur-le-Loir ou les cathédrales de Cahors, d'Auxerre ou d'Albi, mais surtout dans l'art du vitrail comme à la cathédrale de Chartres, de Paris ou de Bourges ou la Sainte-Chapelle à Paris, etc., des enluminures dans les manuscrits depuis l'époqu

In [ ]:
print(len(datac))
with open('dated_events_v2.json', 'w') as outfile:
    json.dump(datac, outfile, indent=4)

In [62]:
import json

datas = open('dated_events_v2.json')
datac = json.load(datas)
print(len(datac))
for d in datac :
    print(d['name'])
    print(d['description'])
    print(d['city'])
    print(d['date'])
    print('')

38
Art_francais
La peinture française est considérée comme une des grandes écoles de peinture par son influence, son histoire et ses productions, aux côtés de la peinture italienne, de la peinture flamande et de la . Les premières manifestations de la peinture sur le territoire français s'observent dans l'art préhistorique des grottes ornées, comme celles de Chauvet vers 35 000 ans AP, Cosquer, Lascaux, Pech Merle, Niaux ou Font-de-Gaume, où sont déjà employé la polychromie, l'estompe, la réserve et même les prémices de la perspective, puis à l'époque romaine avec quelques peintures murales. Au Moyen Âge, on trouve des peintures murales dans les édifices religieux comme Saint-Savin-sur-Gartempe, Nohant-Vic, Tavant, Brinay, Berzé-la-Ville, Montoire-sur-le-Loir ou les cathédrales de Cahors, d'Auxerre ou d'Albi, mais surtout dans l'art du vitrail comme à la cathédrale de Chartres, de Paris ou de Bourges ou la Sainte-Chapelle à Paris, etc., des enluminures dans les manuscrits depuis l'époq

In [ ]:
fcities = open('cities.json')
datac = json.load(fcities)

for d in datac :
    print(d['name'])
    print('')

In [ ]:
for d in datac :
    print(d['name'])
    print(d['description'])
    print(d['city'])
    print(d['date'])
    print('')